## Import Library

In [42]:
import csv
import cv2 
import os
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp 
from sklearn.model_selection import train_test_split
# from sklearn.pipeline import make_pipeline 
# from sklearn.preprocessing import StandardScaler 
# from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.optimizers import Adam

from sklearn.preprocessing import LabelEncoder


## Read Dataset

In [36]:
PATH_DATASET = 'C:/Users/WINDOWS/Downloads/Archve PKM/From anisadewip/dataset bisindo.csv'
dataset2 = 'C:/Users/WINDOWS/Downloads/Archve PKM/From anisadewip/dataset.csv'
df = pd.read_csv(PATH_DATASET)

In [37]:
dept_emp_num = df.groupby('class')['class'].count()
print (dept_emp_num)
df.shape

class
A    2000
B    2000
C    2000
D    2000
E    2000
F    2000
G    2000
H    2000
I    2000
J    2000
K    2000
L    2000
M    2000
N    2000
O    2000
P    2000
Q    3972
R    4298
S    2813
T    3181
U    2966
V    4105
W    2900
X    2998
Y    3589
Z    2600
Name: class, dtype: int64


(65422, 2005)

In [38]:
n_class = len(df['class'].unique())
n_class

26

## Data Preparation


In [39]:
startline, endline = 52000, 52053 
filename1= PATH_DATASET
with open(filename1, 'r', newline='') as f:
    content = [row for i,row in enumerate(csv.reader(f), 1)
                    if i not in range(startline, endline+1)]

filename2 = dataset2
with open(filename2, 'w', newline='') as f:
    csv.writer(f).writerows(content)

print('lines deleted')

lines deleted


In [40]:
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

In [44]:
# Menyimpan objek LabelEncoder ke dalam file menggunakan pickle
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [45]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0,0.493020,0.254202,-1.268193,1.0,0.520213,0.202350,-1.230684,0.999992,0.535368,...,-0.008968,0.0,0.550890,0.193628,0.005945,0.0,0.554839,0.187497,0.006164,0.0
1,0,0.492288,0.254317,-0.771564,1.0,0.519469,0.202352,-0.737610,0.999992,0.535065,...,-0.008998,0.0,0.544896,0.188848,0.006885,0.0,0.548581,0.184737,0.006941,0.0
2,0,0.491509,0.254290,-0.577080,1.0,0.518755,0.202182,-0.541947,0.999993,0.534694,...,-0.008195,0.0,0.544459,0.186326,0.007189,0.0,0.548009,0.182622,0.007158,0.0
3,0,0.490956,0.253844,-0.715527,1.0,0.518034,0.201477,-0.687629,0.999993,0.534279,...,-0.008701,0.0,0.544136,0.187739,0.007291,0.0,0.547727,0.183628,0.007348,0.0
4,0,0.490460,0.253458,-0.652642,1.0,0.517399,0.200943,-0.621570,0.999994,0.533826,...,-0.008089,0.0,0.543521,0.187384,0.007735,0.0,0.547105,0.183345,0.007777,0.0


In [46]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
65417,25,0.595709,0.274770,-1.042241,0.999976,0.621691,0.215598,-0.991523,0.999845,0.639878,...,-0.009688,0.0,0.646705,0.213423,0.006357,0.0,0.650734,0.208075,0.006525,0.0
65418,25,0.595583,0.272474,-1.054443,0.999977,0.621621,0.213176,-0.997595,0.999855,0.639724,...,-0.008872,0.0,0.646440,0.212107,0.007809,0.0,0.650587,0.206635,0.008038,0.0
65419,25,0.595596,0.270845,-1.083339,0.999978,0.621621,0.211833,-1.018282,0.999862,0.639649,...,-0.008821,0.0,0.645642,0.210429,0.007302,0.0,0.649870,0.204948,0.007459,0.0
65420,25,0.595181,0.268767,-1.083782,0.999980,0.621220,0.210127,-1.018751,0.999870,0.639161,...,-0.008805,0.0,0.645396,0.208938,0.007480,0.0,0.649592,0.203283,0.007668,0.0
65421,25,0.593558,0.268589,-1.108534,0.999981,0.620092,0.209301,-1.048589,0.999877,0.638199,...,-0.008696,0.0,0.645565,0.207431,0.007687,0.0,0.649840,0.201794,0.007878,0.0


In [47]:
df[df['class']==0]

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0,0.493020,0.254202,-1.268193,1.000000,0.520213,0.202350,-1.230684,0.999992,0.535368,...,-0.008968,0.0,0.550890,0.193628,0.005945,0.0,0.554839,0.187497,0.006164,0.0
1,0,0.492288,0.254317,-0.771564,1.000000,0.519469,0.202352,-0.737610,0.999992,0.535065,...,-0.008998,0.0,0.544896,0.188848,0.006885,0.0,0.548581,0.184737,0.006941,0.0
2,0,0.491509,0.254290,-0.577080,1.000000,0.518755,0.202182,-0.541947,0.999993,0.534694,...,-0.008195,0.0,0.544459,0.186326,0.007189,0.0,0.548009,0.182622,0.007158,0.0
3,0,0.490956,0.253844,-0.715527,1.000000,0.518034,0.201477,-0.687629,0.999993,0.534279,...,-0.008701,0.0,0.544136,0.187739,0.007291,0.0,0.547727,0.183628,0.007348,0.0
4,0,0.490460,0.253458,-0.652642,1.000000,0.517399,0.200943,-0.621570,0.999994,0.533826,...,-0.008089,0.0,0.543521,0.187384,0.007735,0.0,0.547105,0.183345,0.007777,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0.495647,0.288175,-1.070947,0.999997,0.517610,0.231689,-1.027807,0.999994,0.532386,...,-0.009669,0.0,0.553205,0.229072,0.007321,0.0,0.556772,0.224951,0.007457,0.0
1996,0,0.498159,0.288336,-1.144559,0.999997,0.519529,0.232127,-1.092978,0.999994,0.533568,...,-0.009745,0.0,0.553975,0.229567,0.007261,0.0,0.557522,0.225435,0.007373,0.0
1997,0,0.499476,0.288395,-1.143304,0.999997,0.520317,0.232362,-1.092597,0.999993,0.534007,...,-0.009699,0.0,0.553795,0.230379,0.007563,0.0,0.557332,0.226371,0.007709,0.0
1998,0,0.493127,0.265161,-1.037224,0.999981,0.516977,0.211771,-0.966226,0.999982,0.532918,...,-0.008482,0.0,0.546084,0.195277,0.009557,0.0,0.550297,0.189528,0.009934,0.0


## Split Data

In [48]:
X = df.drop('class', axis=1) # features akan menjadi semua nilai koord
y = df['class'] # target = label kelas
y = to_categorical(y).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
58595,0.519896,0.294245,-1.339151,0.999969,0.542441,0.232559,-1.290504,0.999873,0.556987,0.234395,...,-0.009868,0.0,0.571430,0.227354,0.006067,0.0,0.574930,0.222752,0.006131,0.0
56056,0.533113,0.294864,-1.050633,0.999983,0.556905,0.237914,-1.012291,0.999800,0.572745,0.241678,...,-0.008762,0.0,0.587013,0.240073,0.008707,0.0,0.590443,0.235917,0.008885,0.0
1614,0.498124,0.268291,-1.150237,0.999998,0.518559,0.213160,-1.098981,0.999993,0.533176,0.213549,...,-0.010371,0.0,0.549678,0.206668,0.007177,0.0,0.553201,0.202300,0.007337,0.0
16611,0.548358,0.280008,-1.038380,0.999991,0.568576,0.222048,-0.995106,0.999913,0.586179,0.222783,...,-0.010280,0.0,0.601150,0.219764,0.007195,0.0,0.604503,0.215940,0.007282,0.0
48844,0.527541,0.289707,-1.105359,0.999988,0.548729,0.232197,-1.056798,0.999906,0.562425,0.232938,...,-0.008585,0.0,0.581383,0.239950,0.008282,0.0,0.585251,0.235392,0.008382,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11110,0.536827,0.268795,-0.644200,0.999961,0.561302,0.216108,-0.607818,0.999842,0.579284,0.217198,...,-0.010276,0.0,0.594867,0.203549,0.005777,0.0,0.598526,0.199405,0.005783,0.0
59064,0.530693,0.283121,-1.293649,0.999989,0.553605,0.224963,-1.239142,0.999912,0.566238,0.227613,...,-0.008306,0.0,0.579290,0.215940,0.008641,0.0,0.582722,0.212059,0.008763,0.0
27983,0.482116,0.307991,-0.923315,1.000000,0.501053,0.242906,-0.898550,0.999999,0.515988,0.240984,...,-0.009814,0.0,0.532027,0.239732,0.007686,0.0,0.535458,0.235537,0.007833,0.0
62397,0.498415,0.266114,-1.075532,0.999996,0.524303,0.207532,-1.028025,0.999990,0.539929,0.210503,...,-0.009803,0.0,0.549005,0.206356,0.004573,0.0,0.552637,0.201829,0.004548,0.0


In [49]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [50]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [51]:
print('X_test', X_test.shape)
print('X_train', X_train.shape)
print('y_test', y_test.shape)
print('y_train', y_train.shape)

X_test (19627, 2004)
X_train (45795, 2004)
y_test (19627, 26)
y_train (45795, 26)


## Training Model


In [52]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [53]:
model = Sequential([
    # LSTM(128, return_sequences=True, activation='relu'),
    Dense(52, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(130, activation='relu'),
    Dense(104, activation='relu'),
    Dense(52, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

In [54]:
# model = Sequential()
# # model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(2004)))
# # model.add(LSTM(128, return_sequences=True, activation='relu'))
# # model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu', input_shape=(2004,n_class)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(n_class, activation='softmax'))

In [55]:
optimizer = Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer, 
    loss='categorical_crossentropy', 
    metrics=['categorical_accuracy']
)

In [57]:
model.fit(X_train, y_train, epochs=100, batch_size=24)

Epoch 1/100
1909/1909 [==============================] - 8s 4ms/step - loss: 2.4797 - categorical_accuracy: 0.2068
Epoch 2/100
1909/1909 [==============================] - 7s 3ms/step - loss: 1.7969 - categorical_accuracy: 0.3752
Epoch 3/100
1909/1909 [==============================] - 6s 3ms/step - loss: 1.4696 - categorical_accuracy: 0.5077
Epoch 4/100
1909/1909 [==============================] - 6s 3ms/step - loss: 1.1981 - categorical_accuracy: 0.6092
Epoch 5/100
1909/1909 [==============================] - 7s 4ms/step - loss: 0.9695 - categorical_accuracy: 0.6812
Epoch 6/100
1909/1909 [==============================] - 7s 4ms/step - loss: 0.8196 - categorical_accuracy: 0.7227
Epoch 7/100
1909/1909 [==============================] - 7s 4ms/step - loss: 0.7322 - categorical_accuracy: 0.7474
Epoch 8/100
1909/1909 [==============================] - 8s 4ms/step - loss: 0.6682 - categorical_accuracy: 0.7644
Epoch 9/100
1909/1909 [==============================] - 9s 5ms/step - loss: 0.6

In [58]:
# Evaluasi model pada data uji
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

614/614 - 2s - loss: 0.0484 - categorical_accuracy: 0.9894 - 2s/epoch - 4ms/step

Test accuracy: 0.9893513917922974


In [59]:
# Menyimpan model yang telah ditraining
model.save('model-v_anisa.h5')

In [60]:
model

## Live Testing

In [ ]:
# Menggunakan objek LabelEncoder dari file
with open('label_encoder.pkl', 'rb') as file:
    loaded_encoder = pickle.load(file)

In [ ]:
# Meload model yang telah ditraining
model = load_model('model-v_anisa.h5')
model

In [61]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

In [62]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand                                  
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export koordinat
        try:
            print('\nmasuk proses\n----------------')
            # Mengambil Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Mengambil Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Menyatukan baris
            row = pose_row+face_row
            print('proses row\n----------------')
            
            
            X = pd.DataFrame([row])
            print('proses predict\n----------------')
            prediction = model.predict(X)[0]
            body_language_class = np.argmax(prediction)
            body_language_prob = prediction[body_language_class]
            body_language_class = label_encoder.inverse_transform(np.array([body_language_class]))[0]
            print(f'body_language_class: {body_language_class}')
            print(f'body_language_prob: {body_language_prob}\n')
            
            # Ambil koordinat telinga
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            print('proses coords\n----------------')
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            print('proses gambar\n----------------')
            
            # status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display prob
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob,2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------
proses row
----------------
proses predict
----------------
1/1 [==============================] - 0s 134ms/step
body_language_class: O
body_language_prob: 0.9999995231628418

proses coords
----------------
proses gambar
----------------

masuk proses
----------------
proses row
----------------
proses predict
----------------
1/1 [==============================] - 0s 50ms/step
body_language_class: T
body_language_prob: 0.9691906571388245

proses coords
----------------
proses gambar
----------------

masuk proses
----------------
proses row
----------------
proses predict
----------------
1/1 [==============================] - 0s 47ms/step
body_language_class: W
body_language_prob: 0.9968492388725281

proses coords
----------------
proses gambar
----------------

masuk pr

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000248CA490AF0>
Traceback (most recent call last):
  File "C:\Users\WINDOWS\AppData\Local\Programs\Python\Python39\lib\weakref.py", line 371, in remove
    self = selfref()
KeyboardInterrupt: 


proses predict
----------------
1/1 [==============================] - 0s 42ms/step
body_language_class: O
body_language_prob: 0.9498947858810425

proses coords
----------------
proses gambar
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------

masuk proses
----------------
proses row
----------------
proses predict
----------------
1/1 [==============================] - 0s 42ms/step
body_language_class: T
body_language_prob: 0.9999678134918213

proses coords
----------------
proses gambar
----------------

masuk proses
----------------
proses row
----------------
proses predict
----------------
1/1 [==============================] - 0s 39ms/step
body_language_class: W
body_language_prob: 0.9999567270278931

proses coords
----------------
proses gambar
----------------

masuk proses
----------------
proses row
----------------
proses predict
----------------
1/1 [====

In [ ]:
body_language_class


13

In [ ]:
X.shape
prediction = model.predict(X)[0]
class_predict = np.argmax(prediction)
class_predict

1/1 [==============================] - 0s 44ms/step


13

In [ ]:
prediction

array([2.0557889e-37, 5.2379737e-24, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.2392048e-34, 6.1147559e-01, 2.1426574e-19, 4.3748639e-20,
       1.3094006e-34, 2.9011444e-12, 1.9911860e-15, 9.3840508e-12,
       1.0223558e-20, 5.9972801e-05, 3.8450009e-01, 1.5108255e-24,
       3.9643459e-03, 1.0009348e-12], dtype=float32)

In [ ]:
class_prob = prediction[class_predict]
class_prob

0.6114756

In [ ]:
res_class = label_encoder.inverse_transform(np.array([class_predict]))
res_class

NameError: name 'label_encoder' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()